In [ ]:
## mashup script for batch changes
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [ ]:
# variables
with open("/credentials.json") as c:
	credentials = json.load(c)

fulcrum = Fulcrum(key=credentials['fulcrum_api'])

osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 5000

In [ ]:
# target data
formIDTarget = "0092aa7f-a2e8-4972-8f26-46eb98b23a12"  #change this to the app you want to update
formdataTarget = fulcrum.forms.find(formIDTarget)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountTarget = fulcrum.forms.find(formIDTarget)['form']['record_count']
pagesTarget = math.ceil(recordCountTarget / recordsPerPage)

In [ ]:
## get target data
if pagesTarget > 1:
	for p in range(1, pagesTarget + 1):
		dataPage = fulcrum.records.search(url_params={'form_id': formIDTarget, 'page': p, 'per_page': recordsPerPage})[
			'records']
		if p > 1:
			dataTarget.extend(dataPage)
		else:
			dataTarget = dataPage
else:
	dataTarget = fulcrum.records.search(url_params={'form_id': formIDTarget, 'page': 1, 'per_page': recordsPerPage})[
		'records']

In [ ]:
# set up counters
noloc = missmatch = match = notfilter = incomplete = 0
rfi = nar = match = oh = incomplete = 0

Check out all project in this organisation (under this API)

In [ ]:
# get unique project IDs in a list
res = list()
for record in dataTarget:
    project_id = record['project_id']
    if project_id not in res:
            res.append(project_id)


In [ ]:
# create dict with matching names
result = {}
for i in res:
    try:
        project_info = fulcrum.projects.find(i)['project']
        result[i] = project_info['name']

    except:
        project_info = 'Unknown Project'
        result['None'] = 'Unknown'
print(result)

Option one: fill the project field (if empty) with the Name column of the KML

In [ ]:
# read a KMl file with a Name column
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file('AMM.kml', driver='KML')

In [ ]:
for record in dataTarget:
    if record['project'] == False:
        match = match + 1
        pdf = pd.DataFrame(
            {'ID': ['1'],
             'Latitude': record['latitude'],
             'Longitude': record['longitude']})

        gdf = gpd.GeoDataFrame(
            pdf, geometry=gpd.points_from_xy(pdf.Longitude, pdf.Latitude),crs=df.crs)

        pointInPolys = gpd.tools.sjoin(gdf, df, predicate="within", how='left')
        record['project'] = [pointInPolys['Name'][0]]
        recordUpdated = fulcrum.records.update(record['id'], record)
        print(match)
    else:
        ##print('other')
        missmatch = missmatch + 1

Option two: Manually update based on a

In [ ]:
# read geodata and plot
poly = r'/Users/philipp/Downloads/Received-projects-translated.gpkg' #SHP, geojson
polydata = gpd.read_file(poly)
polydata = polydata.reset_index()
polydata = polydata.explode(index_parts=True) #multiparts to single parts
polydata = polydata.reset_index()
polydata['p'] = polydata.index

polydata.plot('p',figsize=[10,10], legend=True, categorical=True,label='p')

In [ ]:
# buffer, maybe adjust
polydata['geometry']=polydata.buffer(0.0005) #WGS projection is in decimal degree: 0.5=1km

In [ ]:
# select polygon based on index id
polygon = polydata.loc[0,'geometry']
polygon

In [ ]:
# check if points are in polygon and assign new project ID based on the list above
for record in dataTarget:
    point = Point(record['longitude'], record['latitude'])
    try:
        if polygon.contains(point):
            # (str(record['project_id']) not in '7739a1cf-2bfd-4a10-ad53-094150492232') and
            record['project_id'] = '7739a1cf-2bfd-4a10-ad53-094150492232'
            recordUpdated = fulcrum.records.update(record['id'], record)
            match = match + 1
            print('in:',match)
        else:
            missmatch=missmatch+1
            print('out:',missmatch)
    except KeyError:
        noproject = noproject + 1